# GPT-5.1 Apply Patch Tool Example

This notebook demonstrates how to use the **apply_patch** tool with GPT-5.1 via OpenAI's Responses API. The apply_patch tool enables agents to create, update, and delete files using structured diffs, making it ideal for code editing tasks.

AUTHOR: [Priyanshu Deshmukh](https://github.com/priyansh4320)

## Overview

The `apply_patch` tool is a built-in capability in GPT-5.1 that allows agents to:
- **Create files**: Generate new files with specified content
- **Update files**: Modify existing files using unified diff format
- **Delete files**: Remove files from the workspace

Unlike traditional code execution methods, the apply_patch tool provides structured, controlled file operations that are safer and more precise than raw code generation.

## Requirements

AG2 requires `Python>=3.10`. To run this notebook, you need:
- GPT-5.1 access (currently in beta)
- OpenAI API key
- AG2 installed with OpenAI support


pip install ag2[openai]
```

For more information, please refer to the [installation guide](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/installing-ag2).

In [ ]:
# Install AG2 if needed
# %pip install ag2[openai]

## Configuration

Set up your OpenAI API key and configure the LLM to use the Responses API with GPT-5.1.

In [3]:
import os
from pathlib import Path

from autogen import ConversableAgent, LLMConfig

llm_config = LLMConfig(
    config_list={
        "api_type": "responses",
        "model": "gpt-5.1",
        "api_key": os.getenv("OPENAI_API_KEY"),
        "built_in_tools": ["apply_patch"],
    },
)

## Create the Agent

Create a coding assistant agent that can use the apply_patch tool. The tool is automatically available when you specify it in `built_in_tools`.

In [2]:
# Create a coding assistant agent
coding_agent = ConversableAgent(
    name="coding_assistant",
    llm_config=llm_config,
    system_message="""You are a helpful coding assistant. You can create, edit, and delete files
    using the apply_patch tool. When making changes, always use the apply_patch tool rather than
    writing raw code blocks. Be precise with your file operations and explain what you're doing.""",
)

## Example 1: Creating a New Project

Let's start by creating a simple Python project with multiple files.

In [4]:
import re

class _V4ADiffApplier:
    """Minimal V4A diff interpreter with no external deps."""

    __slots__ = ("_original", "_cursor", "_result")
    _HUNK_RE = re.compile(r"^@@ -(\d+)(?:,(\d+))? \+(\d+)(?:,(\d+))? @@")

    def __init__(self, original_text: str):
        self._original = original_text.splitlines()
        self._cursor = 0
        self._result: list[str] = []

    # ---- public -----------------------------------------------------------
    def apply(self, diff: str, create: bool) -> str:
        if create or not self._original:
            return self._reconstruct_from_create(diff)

        lines = diff.splitlines()
        idx = 0
        while idx < len(lines):
            header = lines[idx]
            match = self._HUNK_RE.match(header)
            if not match:
                idx += 1
                continue

            old_start = int(match.group(1))
            idx += 1
            self._emit_unchanged_until(old_start - 1)

            while idx < len(lines) and not lines[idx].startswith("@@"):
                self._consume_hunk_line(lines[idx])
                idx += 1

        self._result.extend(self._original[self._cursor :])
        return "\n".join(self._result)

    # ---- private ---------------------------------------------------------
    def _reconstruct_from_create(self, diff: str) -> str:
        new_lines: list[str] = []
        for line in diff.splitlines():
            if not line:
                new_lines.append("")
            elif line.startswith(("@@", "---", "+++")):
                continue
            elif line.startswith("+"):
                new_lines.append(line[1:])
            elif line.startswith("-"):
                continue
            else:
                new_lines.append(line)
        return "\n".join(new_lines)

    def _emit_unchanged_until(self, target_line: int) -> None:
        while self._cursor < target_line and self._cursor < len(self._original):
            self._result.append(self._original[self._cursor])
            self._cursor += 1

    def _consume_hunk_line(self, line: str) -> None:
        if not line or line.startswith("\\ No newline"):
            return
        prefix = line[0] if line else " "
        payload = line[1:] if len(line) > 1 and prefix in "+- " else line

        if prefix == "+":
            self._result.append(payload)
        elif prefix == "-":
            if self._cursor >= len(self._original):
                raise ValueError(f"Deletion beyond file end at line {len(self._result) + 1}")
            if payload != self._original[self._cursor]:
                raise ValueError(
                    f"Deletion mismatch at line {self._cursor + 1}:\n"
                    f"  Expected: {self._original[self._cursor]!r}\n"
                    f"  Got:      {payload!r}"
                )
            self._cursor += 1
        else:  # context (' ' or no prefix)
            if self._cursor < len(self._original):
                if payload != self._original[self._cursor]:
                    raise ValueError(
                        f"Context mismatch at line {self._cursor + 1}:\n"
                        f"  Expected: {self._original[self._cursor]!r}\n"
                        f"  Got:      {payload!r}"
                    )
                self._result.append(payload)
                self._cursor += 1
            else:
                self._result.append(payload)



In [5]:
def apply_diff(current_content: str, diff: str, create: bool = False) -> str:
    """Apply a V4A diff to file content."""
    applier = _V4ADiffApplier(current_content)
    return applier.apply(diff, create)

In [47]:
editor= WorkspaceEditor(workspace_dir=os.getcwd())
editor.create_file(operation={"path": "test.py", "diff": """
+class Calculator:
+   
+
+    def add(self, a, b):
+ 
+        return a + b
"""
})

workspace_dir /Users/priyanshu/Documents/GitHub/ag2/notebook
test.py
matched
broke loop
full_path
workspace_dir /Users/priyanshu/Documents/GitHub/ag2/notebook
full_path: /Users/priyanshu/Documents/GitHub/ag2/notebook/test.py
success:
content: 
class Calculator:
   

    def add(self, a, b):
 
        return a + b
success 


{'status': 'completed', 'output': 'Created test.py'}

In [ ]:
# Create a new project structure
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    Create a new Python project folder called 'calculator' with the following structure:
    1. Create a main.py file with a Calculator class that has methods for add, subtract, multiply, and divide
  
    """,
    max_turns=3,
    clear_history=True,
)

print("Chat Summary:")
print(result.summary)

coding_assistant (to coding_assistant):


    Create a new Python project folder called 'calculator' with the following structure:
    1. Create a main.py file with a Calculator class that has methods for add, subtract, multiply, and divide

    

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
coding_assistant (to coding_assistant):

<apply_patch_call: create_file on calculator/main.py (status: completed) diff: +class Calculator:
+    """A simple calculator class providing basic arithmetic operations."""
+
+    def add(self, a, b):
+        """Return the sum of a and b."""
+        return a + b
+
+    def subtract(self, a, b):
+        """Return the result of subtracting b from a."""
+        return a - b
+
+    def multiply(self, a, b):
+        """Return the product of a and b."""
+        return a * b
+
+    def divide(self, a, b):
+        """Return the result of dividing a by b.
+
+        Raises:
+            ZeroDiv

## Example 2: Modifying Existing Files

Now let's update the calculator to add more features.

In [ ]:
# Update the calculator with new features
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    In calculator folder, add power and square root methods to the Calculator class in main.py
    """,
    max_turns=2,
)

print("Chat Summary:")
print(result.summary)

## Example 3: Refactoring Code

Let's refactor the code to improve its structure.

In [ ]:
# Refactor the calculator code
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    Refactor the calculator code in calculator folder:
    1. Split the Calculator class into separate files: basic_operations.py and advanced_operations.py
    2. Update main.py to import from both modules
    3. Update tests to reflect the new structure
    """,
    max_turns=6,
)

print("Chat Summary:")
print(result.summary)

## Example 4: Creating a Complete Application

Let's create a more complex application: a simple web API using FastAPI.

In [ ]:
# Create a FastAPI application
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    Create a FastAPI application with the following structure:
    1. Create app/main.py with a FastAPI app and a simple /health endpoint
    2. Create app/__init__.py
    3. Create app/requirements.txt with fastapi and uvicorn
    4. Create a app/README.md with setup and run instructions
    5. Create app/.gitignore file for Python projects
    """,
    max_turns=2,
    clear_history=True,
)

print("Chat Summary:")
print(result.summary)

## Example 5: Bug Fixing

The agent can also fix bugs in existing code. Let's first create some code with a bug, then ask the agent to fix it.

In [ ]:
# First, create a file with a bug
buggy_code = """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)  # Bug: doesn't handle empty list

def divide(a, b):
    return a / b  # Bug: doesn't handle division by zero
"""

# Write the buggy file manually (or use the agent)
Path("buggy_math.py").write_text(buggy_code)

In [ ]:
# Now ask the agent to fix the bugs
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    I have a file called buggy_math.py with some bugs:
    1. The calculate_average function doesn't handle empty lists
    2. The divide function doesn't handle division by zero
    Please fix both bugs by adding proper error handling.
    """,
    max_turns=2,
    clear_history=True,
)

print("Chat Summary:")
print(result.summary)

## Example 6: Work with a dedicated Workspace Directory

The Configurations allows a user to create a dedicated workspace_dir for themselves which serves as a root project directory. 

In [ ]:
llm_config = LLMConfig(
    config_list={
        "api_type": "responses",
        "model": "gpt-5.1",
        "api_key": os.getenv("OPENAI_API_KEY"),
        "built_in_tools": ["apply_patch"],
        "workspace_dir": "./my_project_folder",  # NEW: Just specify workspace_dir here!
    },
)

# Create agent - no need to manually create editor or patch_tool
coding_agent = ConversableAgent(
    name="coding_assistant",
    llm_config=llm_config,
    system_message="""You are a helpful coding assistant...""",
)

# Tool is automatically registered! Just use it:
result = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="""
    Create app.py in the workspace directory,
    create a app.yaml file,
    create a app.sh file,
    create a folder /tests,
    create tests/test_app.py
    """,
    max_turns=2,
)

# Example 8: Working With Allowed Paths

The  configuration introduces allowed_paths:
List of allowed path patterns (for security).
                Supports glob-style patterns with ** for recursive matching.
                Works for both local filesystem and cloud storage paths.

                Examples:
                    - ["**"] - Allow all paths (default)
                    - ["src/**"] - Allow all files in src/ and subdirectories
                    - ["my-bucket/**"] - Allow all paths in cloud storage bucket
                    - ["s3://my-bucket/src/**"] - Allow paths in S3 bucket

In [ ]:
# Configure LLM with workspace_dir and allowed_paths
llm_config = LLMConfig(
    config_list={
        "api_type": "responses",
        "model": "gpt-5.1",
        "api_key": os.getenv("OPENAI_API_KEY"),
        "built_in_tools": ["apply_patch"],
        "workspace_dir": "./my_project_folder",
        "allowed_paths": ["src/**", "*.py", "tests/**"],  # Only allow operations in these paths
    },
)

# Create agent - no need to manually create editor or patch_tool
coding_agent = ConversableAgent(
    name="coding_assistant",
    llm_config=llm_config,
    system_message="""You are a helpful coding assistant...""",
)

Test 1: Try to create file in allowed path (should work)

In [ ]:
result1 = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="Create src/main.py with a simple hello world function",
    max_turns=2,
)

Test 2: Try to create file in NOT allowed path (should fail with clear error)

In [ ]:
result2 = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="Create config/settings.json in the config directory (outside of src/ and tests/)",
    max_turns=2,
)

Test 3: Try to update file in NOT allowed path

In [ ]:
result3 = coding_agent.initiate_chat(
    recipient=coding_agent,
    message="Create a file called root_file.py in the root of the workspace (not in src/ or tests/)",
    max_turns=2,
)

## Understanding the Apply Patch Operations

The apply_patch tool uses three types of operations:

1. **create_file**: Creates a new file with the specified content
2. **update_file**: Updates an existing file using unified diff format
3. **delete_file**: Deletes a file from the workspace

### Diff Format

The update_file operation uses unified diff format. Here's an example:

```
@@ -1,3 +1,3 @@
 def hello():
-    print("World")
+    print("Hello, World!")
     return True
```

This format is generated automatically by GPT-5.1 when using the apply_patch tool.

## Best Practices

1. **Start with Clear Instructions**: Provide detailed requirements for what you want to create or modify

2. **Review Changes**: Always review the files created or modified by the agent before using them in production

4. **Iterative Development**: Break complex tasks into smaller steps and verify each step before proceeding

5. **Test Your Code**: Always test the generated code to ensure it works as expected

6. **Handle Errors Gracefully**: The agent can fix bugs, but it's good practice to review error messages carefully

## Troubleshooting

### Common Issues

1. **File Not Found Errors**: Make sure the file path is correct relative to the workspace directory

2. **Permission Errors**: Ensure the agent has write permissions to the workspace directory

3. **Invalid Diff Format**: If you manually create diffs, ensure they follow the unified diff format correctly

4. **API Errors**: Verify your OpenAI API key has access to GPT-5.1

### Getting Help

For more information, check:
- [AG2 Documentation](https://docs.ag2.ai)
- [OpenAI Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [GitHub Issues](https://github.com/AG2ai/ag2/issues)

## Next Steps

Now that you understand how to use the apply_patch tool, you can:

- Create more complex applications
- Integrate with other tools and agents
- Build automated code generation workflows
- Experiment with different approval mechanisms

Happy coding! 🚀